# **Computational Theory Assessment G00417529**

# **Imports**

In [733]:
# numpy and unittest imports
import numpy as np
import unittest

# 32-bit unsigned integer alies to reduce verbosity of notebook
u32 = np.uint32

# Used for benchmarking function performance
import timeit

## **Introduction**
This section defines technical terms used for all the solutions in this notebook

### **Bitwise Terms and Operations**
**Bitwise Operations** are actions directly applied to individual bits of binary numbers. <br>
**Binary Words** are the natural unit of data a computer CPU can process with fixed lengths like 32bits (4 bytes) or 64 bits (8 bytes) <br><br>

**Bitwise AND ($\land$)/(`&`)**<br>
Compares each bit of two **binary words**.

For every bit position:
- The result bit is `1` only if both corresponding bits in the operands are `1`.
- Otherwise, the result bit is `0`.

<table style="border: 1px solid black; border-collapse: collapse; text-align: center; padding: 8px;">
  <tr>
    <th style="border: 1px solid black;">X</th>
    <th style="border: 1px solid black;">Y</th>
    <th style="border: 1px solid black;">X & Y</th>
  </tr>
  <tr>
    <td style="border: 1px solid black;">0</td>
    <td style="border: 1px solid black;">0</td>
    <td style="border: 1px solid black;">0</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">0</td>
    <td style="border: 1px solid black;">1</td>
    <td style="border: 1px solid black;">0</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">1</td>
    <td style="border: 1px solid black;">1</td>
    <td style="border: 1px solid black;">1</td>
  </tr>
</table><br>

**Bitwise OR (`|`)**<br>
For every bit position:
- The bit result is `1` if either bit is `1`.
- Otherwise, the result bit is `0`.

<table style="border: 1px solid black; border-collapse: collapse; text-align: center; padding: 8px;">
  <tr>
    <th style="border: 1px solid black;">X</th>
    <th style="border: 1px solid black;">Y</th>
    <th style="border: 1px solid black;">X | Y</th>
  </tr>
  <tr>
    <td style="border: 1px solid black;">0</td>
    <td style="border: 1px solid black;">0</td>
    <td style="border: 1px solid black;">0</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">0</td>
    <td style="border: 1px solid black;">1</td>
    <td style="border: 1px solid black;">1</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">1</td>
    <td style="border: 1px solid black;">1</td>
    <td style="border: 1px solid black;">1</td>
  </tr>
</table><br>

**Bitwise XOR ($\oplus$)/(`^`)**<br>
For every bit position:
- The bit result is `1` if both bits are different.
- Otherwise, the result bit is `0`.

<table style="border: 1px solid black; border-collapse: collapse; text-align: center; padding: 8px;">
  <tr>
    <th style="border: 1px solid black;">X</th>
    <th style="border: 1px solid black;">Y</th>
    <th style="border: 1px solid black;">X ^ Y</th>
  </tr>
  <tr>
    <td style="border: 1px solid black;">0</td>
    <td style="border: 1px solid black;">0</td>
    <td style="border: 1px solid black;">0</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">0</td>
    <td style="border: 1px solid black;">1</td>
    <td style="border: 1px solid black;">1</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">1</td>
    <td style="border: 1px solid black;">1</td>
    <td style="border: 1px solid black;">0</td>
  </tr>
</table><br>

**Bitwise NOT (`~`)**<br>
Flips every bit (`0→1`, `1→0`)

<table style="border: 1px solid black; border-collapse: collapse; text-align: center; padding: 8px;">
  <tr>
    <th style="border: 1px solid black;">X</th>
    <th style="border: 1px solid black;">~X</th>
  </tr>
  <tr>
    <td style="border: 1px solid black;">0</td>
    <td style="border: 1px solid black;">1</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">1</td>
    <td style="border: 1px solid black;">0</td>
  </tr>
</table><br>

## **Helper Functions**
Used to avoid duplicate code throughout solutions

### Local Unit Test Runner
Prevents variable redeclaration when running a specific test suite

In [734]:
def run_local_scope_tests(test_class, verbosity_lvl=0):
    """
    Runs specific unittest.TestCase without redeclaring variables in the global scope.

    Parameters:
        test_class : unittest.TestCase
            The test class containing test methods.
        verbosity_lvl : int, optional
            Verbosity level for the test runner (default is 0).

    Returns:
        result : unittest.result.TestResult
            The result object containing information about the tests run.
    """

    # Load all test methods from the given TestCase class
    suite = unittest.TestLoader().loadTestsFromTestCase(test_class)

    # Create a test runner with the specified verbosity
    runner = unittest.TextTestRunner(verbosity=verbosity_lvl)

    # Run the test suite and return the result
    return runner.run(suite)

### Converter
Automatically converts all plain int parameters of a given function into np.uint32 values, eliminating the need to implement individual type-checking logic for each method. <br>
Type checking only needs to be tested here once; instead of per method.

In [735]:
def force_u32(func, *args, **kwargs):
    # --------- Convert positional arguments ---------
    # Iterate through all positional arguments (collected in the `args` tuple)
    # If an argument is a plain Python int, convert it to np.uint32 using the validated converter.
    # Non-int arguments (e.g., np.uint32, floats, strings) are passed through unchanged.
    new_args = tuple(
        u32(a) if isinstance(a, int) else a
        for a in args
    )

    # --------- Convert keyword arguments ---------
    # Similarly process keyword arguments (stored in the `kwargs` dict).
    # Each value is checked and converted if it’s an int.
    new_kwargs = {
        key: u32(v) if isinstance(v, int) else v
        for key, v in kwargs.items()
    }

    return func(*new_args, **new_kwargs)


### Converter (force_u32) Tests


In [736]:
def unconverted_method(x: u32, y: u32) -> u32:
    return x * y

#### Unit Tests

In [737]:
class TestForceU32(unittest.TestCase):
    """Unit tests for @force_u32 decorator and convert_to_u32 utility."""

    def test_aliasing(self):
        """Test that u32 alias returns np.uint32."""
        # Verify that the u32 alias is equivalent to np.uint32
        self.assertEqual(u32(5), np.uint32(5))
        self.assertIsInstance(u32(5), np.uint32)

    def test_positional_args(self):
        """Test that positional arguments are correctly converted by @force_u32."""
        # Call decorated function with positional arguments
        result = force_u32(unconverted_method, 7, 3)

        # Verify the result is correct and of type np.uint32
        self.assertEqual(result, 21)
        self.assertIsInstance(result, np.uint32)

    def test_keyword_args(self):
        """Test that keyword arguments are correctly converted by @force_u32."""
        # Call decorated function with keyword arguments
        result = force_u32(unconverted_method, x=66, y=11)

        # Verify the result is correct and of type np.uint32
        self.assertEqual(result, 726)
        self.assertIsInstance(result, np.uint32)

    # --- Tests to make sure type conversion accounts for boundry values and edge cases ---
    def test_min_value(self):
        """Test conversion of the minimum allowed value '0'."""
        # Call decorated function to automatically convert maximum value
        converted = force_u32(unconverted_method, 0, 0)

        # Test that the converted value is of type np.uint32
        self.assertIsInstance(converted, np.uint32)

        # Test that the converted value matches the expected 0 value
        self.assertEqual(converted, 0)

    def test_max_value(self):
        """Test conversion of the maximum allowed value '2**32 - 1'."""
        # Call decorated function to automatically convert maximum value
        converted = force_u32(unconverted_method, 2**32 - 1, 1)

        # Test that the converted value is of type np.uint32
        self.assertIsInstance(converted, np.uint32)

        # Test that the converted value matches the expected maximum u32 value
        self.assertEqual(converted, 2**32 - 1)

    def test_negative_value(self):
        """Test that negative integers raise a OverflowError."""
        # for each negative value in the list
        for val in [-1, -100, -2**32]:
            # Use subTest for better test reporting on multiple values
            with self.subTest(val=val):
                # Attempt to call decorated function and expect OverflowError
                with self.assertRaises(OverflowError):
                    # if we try to pass this value, it should raise an error for out of bounds
                    # unisgned int 32 can't be a negative
                    force_u32(unconverted_method, val, 1)

    def test_above_max_value(self):
        """Test that integers above the maximum (2**32 - 1) raise a OverflowError."""
        # for each value above the maximum u32
        for val in [2**32, 2**33, 9999999999]:
            # Use subTest for better test reporting on multiple values
            with self.subTest(val=val):
                # Attempt to call decorated function and expect OverflowError
                with self.assertRaises(OverflowError):
                    # if we try to pass this value, it should raise an error
                    # unisgned int 32 exceed its maximum size, while ints in python have a dynamic size 
                    force_u32(unconverted_method, val, 1)


# Run test suite
run_local_scope_tests(TestForceU32)


----------------------------------------------------------------------
Ran 7 tests in 0.000s

OK


<unittest.runner.TextTestResult run=7 errors=0 failures=0>

### Preformance Demo
- Decorator adds overhead for type conversion (x3 slower), but provides safety and readability for user inputs.
- This trade-off is often worthwhile in user-facing code, we prioritise safety and usability over raw performance.
- but for inner loops, or methods where type safety is guaranteed, using undecorated functions is preferred.

In [738]:
def performance_demo():
    """Demonstrate performance difference between decorated and undecorated functions."""
    N = 10**7 # Number of iterations for timing (10^7)
    x, y = 10, 20 # Sample integer inputs

    # Undecorated internal function
    t_internal = timeit.timeit(lambda: unconverted_method(u32(x), u32(y)), number=N)

    # Decorated user-facing function (with conversion)
    t_user = timeit.timeit(lambda: force_u32(unconverted_method, x, y), number=N)

    print(f"Unconverted function time: {t_internal:.4f} s")
    print(f"Converted function time: {t_user:.4f} s")

# Run performance demonstration
# performance_demo()

### Metaclass
Metaclass for decorated method architecture to add default functionality to class without having to define it to reduce verbosity and repetition of said codeblocks when using decorator.

This metaclass abstracts common setup logic for bitwise operation classes,
improving maintainability and consistency while minimizing boilerplate code.
Each class only needs to define its internal function (e.g., __ROTR),
and BitOpMeta automatically provides all other functionality.

In [ ]:
class BitOpMeta(type):
    """
    Metaclass for automatic setup of bitwise operation classes.

    It dynamically:
      • Detects the internal core function (e.g., __ROTR)
      • Creates `.int_safe` function variant
      • Adds a default `assert_me()` test helper if not defined
      • Makes the class directly callable (e.g., ROTR(x, n))
    """

    # IntelliSense-visible placeholder for assert_me
    @staticmethod
    def assert_me(test_case: unittest.TestCase, x: int, *args: int, expected: int, fail_msg: str, **kwargs) -> None:
        """Unit test helper —> dynamically replaced at runtime."""
        raise NotImplementedError("This method is dynamically replaced by BitOpMeta.")

    # IntelliSense-visible placeholder for int_safe
    @staticmethod
    def int_safe(*args: int, **kwargs) -> u32:
        """Type-safe variant —> dynamically replaced at runtime."""
        raise NotImplementedError("This method is dynamically replaced by BitOpMeta.")
    
    def __call__(cls, *args, **kwargs):
        """
        Fires dynamic behaviour from subclass when static class is called for cleaner syntax

        Parameters:
            *args:
                accepts many positional arguments

            *kwargs:
                accepts many key word arguments

        """
        return cls._internal_func(*args, **kwargs)
    
    def __new__(mcls, name, bases, dct):
        """
        Creates the bit operation class at definition time.
        """
        # setting internal function to internal function reference via dictionary fetch
        internal_func = dct[f"_{name}__{name}"]

        # ------------------------------------------------------------------------
        # Dynamically attach an "int_safe" variant of the internal bitwise function.
        #
        # This creates a static method that wraps the low-level (performance-oriented)
        # internal function — typically named like "__ROTR" — with the `force_u32` utility.
        #
        # When called, this wrapper will:
        #   1. Accept any combination of positional (*args) or keyword (**kwargs) arguments.
        #   2. Automatically convert all Python `int` values to NumPy `np.uint32` before execution.
        #   3. Pass the converted arguments to the underlying internal function.
        #
        # The result is a type-safe interface that behaves identically to the internal
        # function but removes the need for manual casting in user-facing code.
        #
        # Example:
        #     ROTR.int_safe(1, 8)
        #     → behaves the same as ROTR(u32(1), u32(8)), ensuring type safety.
        #
        # This definition is created dynamically at class construction time (via metaclass),
        # so every bitwise operation automatically gains an `.int_safe()` variant without
        # redundant boilerplate definitions.
        # ----------------------------------------------------------------------
        dct["int_safe"] = staticmethod(lambda *args, **kwargs: force_u32(internal_func, *args, **kwargs))

        # If the class did not define its own 'assert_me' helper,
        # automatically create a default version.
        # This prevents every bit operation class from having to manually define identical testing helpers.
        if "assert_me" not in dct:
            def assert_me(test_case, x, *args, expected, fail_msg, **kwargs):
                # Fetching int_safe result from method
                result = dct["int_safe"](x, *args, **kwargs)

                # comparing result with expected with expected value
                test_case.assertEqual(result, expected, fail_msg)

            # adding assert_me method to classes dictionary
            # so other functions can see and use it
            dct["assert_me"] = staticmethod(assert_me)

        # Create the new class using the parent metaclass constructor
        cls = super().__new__(mcls, name, bases, dct)

        # we delegated dynamic internal_func behaviour to our __call__ metaclass
        # this essentially allows us to call any class that uses this metaclass as if it was a method
        # with the sudo method behaviour defined in its "internal method" which should be defined as __classname
        # This makes this strucutre syntax cleaner as you only have to call e.g ROTR(x, y) instead of ROTR.fast(x, y)
        cls._internal_func = internal_func

        # return the completed class object at runtime
        return cls


## **Problem 1: Binary Words and Operations**

### 1.1 Parity Function
$\text{Parity}(x, y, z) = x \oplus y \oplus z$ <br>
Implemented as defined in [FIPS, Section 4.1.1](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

**How it works:** <br>
Applies bitwise eclusive or (XOR) to three 32-bit unsigned integers (x, y, z). <br>
For each bit position of x, y and z:
- if an odd number of bits are 1, the result is 1
- if an even number of bits are 1, the result is 0

Worked example shown at [Parity Unit Tests](#1.12-Parity-Unit-Tests) output.

#### 1.11 Design Decisions
**Old design decision**<br>
I chose to statically define all parameter types as NumPy 32-bit unsigned integers ``(np.uint32)`` to avoid writing defensive runtime checks and to improve performance.
In the Secure Hash Standard, this function would be called millions of times per hash computation. While this notebook is primarily meant to demonstrate functionality and could therefore benefit from more flexible, dynamically typed inputs I believe performance is just as critical as correctness in these bitwise operations. After all, these functions are designed to operate at the bit level for efficiency and precision; type checking them with expensive calls would defeat the purpose.

**Updated Design decision**<br>
- The core methods still define parameters with static typing for maximum performance.
- A new type-conversion decorator, ``@force_u32``, provides the best of both worlds; enabling automatic safe conversion of Python int values to ``np.uint32`` where appropriate,<br>
while leveraging existing code. This eliminates the need to maintain duplicate function logic and duplicate test logic (one with type conversion and one with static typing).
- All related methods are now encapsulated in static classes, giving the user explicit control over usage style:
    - ``ClassName()`` High-performance version (expects np.uint32 arguments).
    - ``ClassName.int_safe`` Decorated version with automatic type conversion for readability and convenience.

- Each static class also includes an ``assert_me`` method to streamline and simplify unit testing, improving clarity and reducing boilerplate.
- Methods are dynamically created using metaclass to remove boilderplate code while allowing fast and dynamic functionality

This design allows you to call binary word operations directly with regular Python integers without manually wrapping every argument in ``u32()`` or ``np.uint32()``.<br>
At the same time, performance-critical code (such as within tight loops where functions can be called millions of times) can achieve up to 3× faster execution by using the undecorated internal method.

In [ ]:
class Parity(metaclass=BitOpMeta):
    """
    Operation:
        Applies bitwise eclusive or (XOR) to three 32-bit unsigned integers (x, y, z).
        For each bit position of x, y and z:
            - if an odd number of bits are 1, the result is 1
            - if an even number of bits are 1, the result is 0

    Parameters:
        (all arguments are of numpy unsigned 32-bit integer type)
        x: First integer
        y: Second integer.
        z: Third integer.

    Returns: 
        u32: Operation result
    
    Class Utilities:
        `ROTR.int_safe` runs function with ints as parameters
        `ROTR.assert_me` runs test_case assertEqual function against ``assert_me`` function inputs.
    """

    # interal perfomance function
    @staticmethod
    def __Parity(x: u32, y: u32, z: u32) -> u32:
        # Apply bitwise XOR to the three integers and return the result
        return x ^ y ^ z
    
    # Unit test method template
    # improves readability and reduces boilerplate code in unit tests
    @staticmethod
    def assert_me(test_class: unittest.TestCase, x: int, y: int, z: int, 
               expected: int, fail_msg: str, print_equation: bool = False) -> None:
        """
        Overview:
            Asserts Parity function via TestCase.assertEqual() method.
                - used for readability in unit tests
                - can print detailed equation breakdown for debugging and verification.
                - automatically converts int inputs to np.uint32

        Parameters:
            test_class: The unittest.TestCase instance to use for assertions.
            x: First integer input for Parity.
            y: Second integer input for Parity.
            z: Third integer input for Parity.
            expected: The expected result of Parity(x, y, z).
            fail_msg: Message to display if the test fails.
            print_equation: If True, prints detailed equation breakdown.

        returns:
            None
        """
        
        # fetch result from Parity function
        result: u32 = Parity.int_safe(x, y, z)

        # Optional detailed equation info printout
        if print_equation:
            print("----------------------------------------------------------------------")
            # print formatted test info
            print(f"Testing Parity with inputs: x={x} y={y} z={z}")
            print(f"Equation: {x} ⊕ {y} ⊕ {z} = {result}\n")
            print(f"Expected result: {expected}")
            print(f"Actual result: {result}\n")

            # detailed breakdown of the equation steps
            print("Equation breakdown:")

            # print inputs in binary format
            print(
                f"Inputs as binary:\n"
                f"(x): {x:032b}\n"
                f"(y): {y:032b}\n"
                f"(z): {z:032b}\n"
            )

            # print step 1
            a_result: u32 = x ^ y
            print("Step 1: x ⊕ y:")
            print(f"(x): {x:032b}\n(y): {y:032b}")
            print(f"-------------------------- \n(a): {a_result:032b}\n")

            # print step 2
            b_result: u32 = a_result ^ z
            print("Step 2: a ⊕ z:")
            print(f"(a): {a_result:032b}\n(z): {z:032b}")
            print(f"-------------------------- \nResult: {b_result:032b}\n")

            # end section printout
            print("----------------------------------------------------------------------")

        # Assert that the result matches the expected value
        test_class.assertEqual(result, expected, fail_msg)

### 1.12 Parity Unit Tests

In [741]:
# Unit tests for the Parity method.
class TestParity(unittest.TestCase):
    def test_base_cases(self):
        """Basic XOR combinations."""
        # 0 ⊕ 0 ⊕ 0 = 0
        Parity.assert_me(self, x=0, y=0, z=0, 
                        expected=0, 
                        fail_msg="Failed on all zeros")

        # 1 ⊕ 0 ⊕ 0 = 1
        Parity.assert_me(self, x=1, y=0, z=0, 
                        expected=1, 
                        fail_msg="Failed when only x=1")

        # 1 ⊕ 1 ⊕ 0 = 0
        Parity.assert_me(self, x=1, y=1, z=0,
                        expected=0,
                        fail_msg="Failed on even number of 1s")

        # 1 ⊕ 1 ⊕ 1 = 1
        Parity.assert_me(self, x=1, y=1, z=1, 
                        expected=1,
                        fail_msg="Failed on odd number of 1s")

    def test_non_trivial_cases(self):
        """Non-trivial test cases with larger integers."""

        # Test case 1
        # Showing detailed workings of how Parity method works
        Parity.assert_me(self, x=14, y=32, z=11,
                        expected=37,
                        fail_msg="Failed on non-trivial case 1",
                        print_equation=True)


run_local_scope_tests(TestParity, 2)

test_base_cases (__main__.TestParity.test_base_cases)
Basic XOR combinations. ... ok
test_non_trivial_cases (__main__.TestParity.test_non_trivial_cases)
Non-trivial test cases with larger integers. ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.003s

OK


----------------------------------------------------------------------
Testing Parity with inputs: x=14 y=32 z=11
Equation: 14 ⊕ 32 ⊕ 11 = 37

Expected result: 37
Actual result: 37

Equation breakdown:
Inputs as binary:
(x): 00000000000000000000000000001110
(y): 00000000000000000000000000100000
(z): 00000000000000000000000000001011

Step 1: x ⊕ y:
(x): 00000000000000000000000000001110
(y): 00000000000000000000000000100000
-------------------------- 
(a): 00000000000000000000000000101110

Step 2: a ⊕ z:
(a): 00000000000000000000000000101110
(z): 00000000000000000000000000001011
-------------------------- 
Result: 00000000000000000000000000100101

----------------------------------------------------------------------


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

### 1.2 Ch Function
$\text Ch(x, y, z) = (x \land y) \oplus (\lnot x \land z)$ <br>
Implemented as defined in [FIPS, Section 4.1.1](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

**How it works:**<br>
For each bit position of x, y, and z:
- If the bit in x is 1; take the bit from y
- If the bit in x is 0; take the bit from z

This is achieved by using x as a selector bit mask.
- When a bit in x is 0, the expression $(x \land y)$ evaluates to 0 because the AND operation masks out the corresponding bit in y.
- Simultaneously, (~x & z) evaluates to the corresponding bit in z, because ~x flips the 0 in x to 1, allowing the bit from z to pass through the mask.
- if either one of the expressions evaluates to 1, the $\oplus$ (XOR) will return 1 for that bit.
- This would mean a simple ``OR`` operation would work instead of ``XOR`` as only 1 of the expressions can evaluate to 1 at the same time.

**Worked Example** is shown at [Ch unit tests](#1.21-Cha-Unit-Tests) output.


In [ ]:
class Ch(metaclass=BitOpMeta):
    """
        Operation:
            For each bit position of x, y, and z:
                - If the bit in x is 1; take the bit from y
                - If the bit in x is 0; take the bit from z

        Parameters:
            x (u32): First 32-bit unsigned integer (bit selector mask)
            y (u32): Second 32-bit unsigned integer
            z (u32): Third 32-bit unsigned integer

        Returns:
            u32: Result of the 'Choose' operation.

        Class Utilities:
            `ROTR.int_safe` runs function with ints as parameters
            `ROTR.assert_me` runs test_case assertEqual function against ``assert_me`` function inputs.
        """

    # Internal perfomance function
    @staticmethod
    def __Ch(x: u32, y: u32, z: u32) -> u32:
        return (x & y) ^ (~x & z)

    # Unit test helper method (readable and reusable)
    @staticmethod
    def assert_me(test_class: unittest.TestCase, x: int, y: int, z: int,
                  expected: int, fail_msg: str, print_equation: bool = False) -> None:
        """
        Overview:
            Asserts correctness of the Ch function using unittest's assertEqual().
            Can print detailed equation breakdown for debugging and verification.

        Parameters:
            test_class (unittest.TestCase): The unittest instance performing the assertion.
            x (int): First integer input for Ch.
            y (int): Second integer input for Ch.
            z (int): Third integer input for Ch.
            expected (int): The expected result.
            fail_msg (str): Message to display if the test fails.
            print_equation (bool): If True, prints the detailed bitwise computation.

        Returns:
            None
        """

        # Run the user-facing safe version
        result: u32 = Ch.int_safe(x, y, z)

        # Optional detailed debug print
        if print_equation:
            print("----------------------------------------------------------------------")
            # Print formatted test information
            print(f"Testing Ch with inputs: x:{x} y:{y} z:{z}")
            print(f"Equation: ({x} & {y}) ⊕ (~{x} & {z}) = {result}")
            print(f"Expected result: {expected}")
            print(f"Actual result:   {result}\n")
            
            # Print detailed breakdown of the equation steps
            print("Equation breakdown:")
            print("Inputs as binary:")
            print(f"(x): {x:032b}\n(y): {y:032b}\n(z): {z:032b}\n")

            # Step 1: Compute x & y
            a_result: u32 = x & y
            print("Step 1: x & y:")
            print(f"(x): {x:032b}\n(y): {y:032b}")
            print(f"--------------------------\n(a): {a_result:032b}\n")

            # Step 2: Compute ~x & z
            b_result: u32 = (~x) & z
            print("Step 2: ~x & z:")
            print(f"(~x): {~x & 0xFFFFFFFF:032b}\n(z):  {z:032b}")  # mask to 32 bits
            print(f"--------------------------\n(b):  {b_result:032b}\n")

            # Step 3: XOR results
            final: u32 = a_result ^ b_result
            print("Step 3: (x & y) ⊕ (~x & z):")
            print(f"(a): {a_result:032b}\n(b): {b_result:032b}")
            print(f"--------------------------\nResult: {final:032b}")
            print("----------------------------------------------------------------------")

        # Perform the actual test
        test_class.assertEqual(result, expected, fail_msg)


### 1.21 Cha Unit Tests

In [743]:
# Unit tests for the Ch (Choose) method.
class TestCh(unittest.TestCase):
    def test_base_cases(self):
        """Basic bitwise choose (Ch) combinations."""

        # (0 & 0) ⊕ (~0 & 0) = 0
        Ch.assert_me(self, x=0, y=0, z=0,
                     expected=0,
                     fail_msg="Failed on all zeros")

        # (0 & 0) ⊕ (~0 & 1) = 1
        Ch.assert_me(self, x=0, y=0, z=1,
                     expected=1,
                     fail_msg="Failed when x=0, y=0, z=1")

        # (1 & 0) ⊕ (~1 & 1) = 0
        Ch.assert_me(self, x=1, y=0, z=1,
                     expected=0,
                     fail_msg="Failed when x=1, y=0, z=1")

        # (1 & 1) ⊕ (~1 & 0) = 1
        Ch.assert_me(self, x=1, y=1, z=0,
                     expected=1,
                     fail_msg="Failed when x=1, y=1, z=0")

        # (1 & 1) ⊕ (~1 & 1) = 1
        Ch.assert_me(self, x=1, y=1, z=1,
                     expected=1,
                     fail_msg="Failed when all are 1s")

        # (0 & 1) ⊕ (~0 & 1) = 1
        Ch.assert_me(self, x=0, y=1, z=1,
                     expected=1,
                     fail_msg="Failed when x=0, y=1, z=1")

    def test_non_trivial_cases(self):
        """Non-trivial test cases with larger integers."""

        # Test case 1
        # Showing detailed workings of how cha method works
        Ch.assert_me(self, x=32, y=11, z=42,
                     expected=(32 & 11) ^ (~32 & 42),
                     fail_msg="Failed on non-trivial case 1",
                     print_equation=True)

        # Test case 2
        Ch.assert_me(self, x=63, y=3, z=12,
                     expected=(63 & 3) ^ (~63 & 12),
                     fail_msg="Failed on non-trivial case 2")


run_local_scope_tests(TestCh, 2)

test_base_cases (__main__.TestCh.test_base_cases)
Basic bitwise choose (Ch) combinations. ... ok
test_non_trivial_cases (__main__.TestCh.test_non_trivial_cases)
Non-trivial test cases with larger integers. ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.003s

OK


----------------------------------------------------------------------
Testing Ch with inputs: x:32 y:11 z:42
Equation: (32 & 11) ⊕ (~32 & 42) = 10
Expected result: 10
Actual result:   10

Equation breakdown:
Inputs as binary:
(x): 00000000000000000000000000100000
(y): 00000000000000000000000000001011
(z): 00000000000000000000000000101010

Step 1: x & y:
(x): 00000000000000000000000000100000
(y): 00000000000000000000000000001011
--------------------------
(a): 00000000000000000000000000000000

Step 2: ~x & z:
(~x): 11111111111111111111111111011111
(z):  00000000000000000000000000101010
--------------------------
(b):  00000000000000000000000000001010

Step 3: (x & y) ⊕ (~x & z):
(a): 00000000000000000000000000000000
(b): 00000000000000000000000000001010
--------------------------
Result: 00000000000000000000000000001010
----------------------------------------------------------------------


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

### 1.3 Maj Function
$\text Maj(x, y, z) = (x \land y) \oplus (x \land z) \oplus (y \land z)$ <br>
Implemented as defined in [FIPS, Section 4.1.1](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

**How it works:**<br>
For each bit position of x, y, and z:
- Returns the bit that is in the majority
- if at least two of the inputs are 1

| X | Y | Z |
|---|---|---|
| 1 | 1 | 1 |

$(x \land y) \oplus (x \land z) \oplus (y \land z)$ <br>
**(A):** $(x \land y)$ = $(1 \land 1)$ = 1 <br>
**(B):** $(x \land z)$ = $(1 \land 1)$ = 1 <br>
**(C):** $(y \land z)$ = $(1 \land 1)$ = 1 <br>
**(D):** $ A \oplus B \oplus C $ = $ 1 \oplus 1 \oplus 1 $ = 1

**Worked Example** is shown at [Maj unit tests](#1.31-Maj-Unit-Tests) output.

In [ ]:
class Maj(metaclass=BitOpMeta):
    """
    Operation:
        For each bit position of x, y, and z:
            - Returns the bit that is in the majority
            - if at least two of the inputs are 1

    Parameters:
        x (u32): First 32-bit unsigned integer
        y (u32): Second 32-bit unsigned integer
        z (u32): Third 32-bit unsigned integer

    Returns:
        u32: Result of the 'Majority' operation.

    Class Utilities:
        `ROTR.int_safe` runs function with ints as parameters
        `ROTR.assert_me` runs test_case assertEqual function against ``assert_me`` function inputs.
    """
    
    # Internal perfomance function
    @staticmethod
    def __Maj(x: u32, y: u32, z: u32) -> u32:
        return (x & y) ^ (x & z) ^ (y & z)

    # Unit test helper method (readable and reusable)
    @staticmethod
    def assert_me(test_class: unittest.TestCase, x: int, y: int, z: int,
                  expected: int, fail_msg: str, print_equation: bool = False) -> None:
        """
        Overview:
            Asserts correctness of the Maj function using unittest's assertEqual().
            Can print detailed equation breakdown for debugging and verification.

        Parameters:
            test_class (unittest.TestCase): The unittest instance performing the assertion.
            x (int): First integer input for Maj.
            y (int): Second integer input for Maj.
            z (int): Third integer input for Maj.
            expected (int): The expected result.
            fail_msg (str): Message to display if the test fails.
            print_equation (bool): If True, prints the detailed bitwise computation.

        Returns:
            None
        """

        # Run the user-facing safe version
        result: u32 = Maj.int_safe(x, y, z)

        # Optional detailed debug print
        if print_equation:
            print("----------------------------------------------------------------------")
            # Print formatted test information
            print(f"Testing Maj with inputs: x:{x} y:{y} z:{z}")
            print(f"Equation: (x & y) ⊕ (x & z) ⊕ (y & z) = {result}")
            print(f"Expected result: {expected}")
            print(f"Actual result:   {result}\n")
            
            # Print detailed breakdown of the equation steps
            print("Equation breakdown:")
            print("Inputs as binary:")
            print(f"(x): {x:032b}\n(y): {y:032b}\n(z): {z:032b}\n")

            # Step 1: Compute x & y
            a_result: u32 = x & y
            print("Step 1: x & y:")
            print(f"(x): {x:032b}\n(y): {y:032b}")
            print(f"--------------------------\n(a): {a_result:032b}\n")

            # Step 2: Compute x & z
            b_result: u32 = x & z
            print("Step 2: x & z:")
            print(f"(x): {x:032b}\n(z): {z:032b}")
            print(f"--------------------------\n(b): {b_result:032b}\n")

            # Step 3: Compute y & z
            c_result: u32 = y & z
            print("Step 3: y & z:")
            print(f"(y): {y:032b}\n(z): {z:032b}")
            print(f"--------------------------\n(c): {c_result:032b}\n")

            # Step 4: XOR the intermediate results
            final: u32 = a_result ^ b_result ^ c_result
            print("Step 4: (x & y) ⊕ (x & z) ⊕ (y & z):")
            print(f"(a): {a_result:032b}\n(b): {b_result:032b}\n(c): {c_result:032b}")
            print(f"--------------------------\nResult: {final:032b}")
            print("----------------------------------------------------------------------")

        # Perform the actual test
        test_class.assertEqual(result, expected, fail_msg)


#### 1.31 Maj Unit Tests

In [745]:
# Unit tests for the Maj method.
class TestMaj(unittest.TestCase):
    """Unit tests for the Maj function."""

    def test_base_cases(self):
        """Basic majority validation."""

        # Case 1: x=0, y=0, z=0
        Maj.assert_me(self, x=0, y=0, z=0,
                      expected=0,
                      fail_msg="Failed on all zeros")

        # Case 2: x=0, y=0, z=1
        Maj.assert_me(self, x=0, y=0, z=1,
                      expected=0,
                      fail_msg="Failed when z=1 only")

        # Case 3: x=0, y=1, z=1
        Maj.assert_me(self, x=0, y=1, z=1,
                      expected=1,
                      fail_msg="Failed when x=0, y=1, z=1")

        # Case 4: x=1, y=0, z=1
        Maj.assert_me(self, x=1, y=0, z=1,
                      expected=1,
                      fail_msg="Failed when x=1, y=0, z=1")

        # Case 5: x=1, y=1, z=1
        Maj.assert_me(self, x=1, y=1, z=1,
                      expected=1,
                      fail_msg="Failed when all are 1s")
        
    def test_non_trivial_cases(self):
        """non-trivial majority validation."""

         # Case 1: non-trivial case
        Maj.assert_me(self, x=7, y=8, z=2,
                      expected=2,
                      fail_msg="Failed on large number case",
                      print_equation=True)


# Run tests in a local scope
run_local_scope_tests(TestMaj, 2)


test_base_cases (__main__.TestMaj.test_base_cases)
Basic majority validation. ... ok
test_non_trivial_cases (__main__.TestMaj.test_non_trivial_cases)
non-trivial majority validation. ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


----------------------------------------------------------------------
Testing Maj with inputs: x:7 y:8 z:2
Equation: (x & y) ⊕ (x & z) ⊕ (y & z) = 2
Expected result: 2
Actual result:   2

Equation breakdown:
Inputs as binary:
(x): 00000000000000000000000000000111
(y): 00000000000000000000000000001000
(z): 00000000000000000000000000000010

Step 1: x & y:
(x): 00000000000000000000000000000111
(y): 00000000000000000000000000001000
--------------------------
(a): 00000000000000000000000000000000

Step 2: x & z:
(x): 00000000000000000000000000000111
(z): 00000000000000000000000000000010
--------------------------
(b): 00000000000000000000000000000010

Step 3: y & z:
(y): 00000000000000000000000000001000
(z): 00000000000000000000000000000010
--------------------------
(c): 00000000000000000000000000000000

Step 4: (x & y) ⊕ (x & z) ⊕ (y & z):
(a): 00000000000000000000000000000000
(b): 00000000000000000000000000000010
(c): 00000000000000000000000000000000
--------------------------
Result: 0

<unittest.runner.TextTestResult run=2 errors=0 failures=0>

### 1.4 Implementing Sigma functions
#### 1.41 Helper Functions
The sigma functions use the following methods throughout their implementation:
- **ROTR** (Circular right shift Operation)
- **SHR** ( Right Shift Operation )

These functions have been implemented as defined in [FIPS 180-4, Section 3.2: Operations on Words](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as helper methods to reduce repetiton. 

**Circular right shift Operation**<br>
$\text{ROTR}^n(x) = (x \gg n) \lor (x \ll w - n)$

**Design Descion update**
I've stopped using print_equation flag in assert_me classes, as it's value comes from reasuable code however we only every use it once. so the example of how one would implement reasuable code is already there but from now on 1 example will be done out by hand to reduce the amount of code/comments in each code block


In [ ]:
class ROTR(metaclass=BitOpMeta):
    """
    Right rotate a 32-bit word x by n bits.

    Parameters:
        x (u32): integer to rotate.
        n (u32): Number of bits to rotate ``x``.

    Returns:
        u32: Result of the right rotation.

    Class Utilities:
        `ROTR.int_safe` runs function with ints as parameters
        `ROTR.assert_me` runs test_case assertEqual function against ``assert_me`` function inputs.
    """

    # internal perfomance function
    @staticmethod
    def __ROTR(x: u32, n: u32) -> u32:

        # for w, we use 32 as we are working with a 32 bit unisgned integer
        return (x >> n) | (x << (32 - n))


### ROTR Unit Tests

In [747]:
class TestROTR(unittest.TestCase):
    """Unit tests for the ROTR function."""

    def test_basic_rotations(self):
        """Basic right rotation tests."""
        
        # Case 1: Rotate 0 by any amount (5): result 0
        ROTR.assert_me(self, x=0x0, n=0x5,
                       expected=0x0,
                       fail_msg="Failed rotating 0")

        # Case 2: Rotate 1 by 1: result 0x80000000
        ROTR.assert_me(self, x=0x1, n=0x1,
                       expected=0x80000000,
                       fail_msg="Failed rotating 1 by 1")
        
        # Case 2.1: Same as above, but using direct call
        self.assertEqual(ROTR(u32(0x1), u32(0x1)), 0x80000000, "Failed rotating 1 by 1 directly")

        # case 2.2: same as above, but using safe_int version
        self.assertEqual(ROTR.int_safe(1, 1), 0x80000000, "Failed rotating 1 by 1 via int safe method")

        # Case 3: Rotate 0xFFFFFFFF by 4: result 0xFFFFFFFF
        ROTR.assert_me(self, x=0xFFFFFFFF, n=0x4,
                       expected=0xFFFFFFFF,
                       fail_msg="Failed rotating all ones")

    def test_non_trivial_rotations(self):
        """Non-trivial right rotation tests."""

        # Case 4: Rotate 0x12345678 by 8: result 0x78123456
        ROTR.assert_me(self, x=0x12345678, n=0x8,
                       expected=0x78123456,
                       fail_msg="Failed rotating 0x12345678 by 8")

        # Case 5: Rotate 0xA5A5A5A5 by 16: unchanged
        ROTR.assert_me(self, x=0xA5A5A5A5, n=0x10,
                       expected=0xA5A5A5A5,
                       fail_msg="Failed rotating 0xA5A5A5A5 by 16")

        # Case 6: Fast version check — same as above
        ROTR.assert_me(self, x=0xA5A5A5A5, n=0x10,
                       expected=0xA5A5A5A5,
                       fail_msg="Failed rotating 0xA5A5A5A5 by 16 using fast function version")

# Run tests in a local scope
run_local_scope_tests(TestROTR, 2)


test_basic_rotations (__main__.TestROTR.test_basic_rotations)
Basic right rotation tests. ... ok
test_non_trivial_rotations (__main__.TestROTR.test_non_trivial_rotations)
Non-trivial right rotation tests. ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


RUINN


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

**Right Shift Operation**<br>
$\text{SHR}^n(x) = x \gg n$

In [ ]:
class SHR():
    def __SHR(x: u32, n:u32) -> u32:
        """Logical right shift of x by n bits."""
        return (x >> n)

#### 1.5 large Sigma 0
$\Sigma_{0}^{256}(x) \;=\; \mathrm{ROTR}^2(x) \;\oplus\; \mathrm{ROTR}^{13}(x) \;\oplus\; \mathrm{ROTR}^{22}(x)$

In [749]:
class Sigma0():
    def __Sigma0(x: u32) -> u32:
        return ROTR(x, u32(2)) ^ ROTR(x, u32(13)) ^ ROTR(x, u32(22))
    
    

#### 1.6 Large Sigma 1
$\Sigma_{1}^{256}(x) \;=\; \mathrm{ROTR}^6(x) \;\oplus\; \mathrm{ROTR}^{11}(x) \;\oplus\; \mathrm{ROTR}^{25}(x)$

In [750]:
class Sigma1():
    def __Sigma1(x: u32) -> u32:
        return ROTR(x, u32(6)) ^ ROTR(x, u32(11)) ^ ROTR(x, u32(25))

#### 1.7 Small Sigma 0
$\sigma_{0}^{256}(x) = \mathrm{ROTR}^{7}(x) \;\oplus\; \mathrm{ROTR}^{18}(x) \;\oplus\; \mathrm{SHR}^{3}(x)$

In [751]:
class sigma0():
    def __sigma0(x: u32) -> u32:
        return ROTR(x, u32(7)) ^ ROTR(x, u32(18)) ^ SHR(x, u32(3))

#### 1.8 Small Sigma 1
$\sigma_{1}^{256}(x) = \mathrm{ROTR}^{17}(x) \;\oplus\; \mathrm{ROTR}^{19}(x) \;\oplus\; \mathrm{SHR}^{10}(x)$

In [752]:
class sigma1():
    def __sigma1(x: u32) -> u32:
        return ROTR(x, u32(17)) ^ ROTR(x, u32(19)) ^ SHR(x, u32(10))

## Problem 2: Fractional Parts of Cube Roots

## Problem 3: Padding

## Problem 4: Hashes

## Problem 5: Passwords